In [1]:
import wrds
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
def pull_all_option_price(wrds_username, last_year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	optm_df = []

	for year in range(1996,last_year+1):
		sql_query = f"""
						SELECT 
							a.date, a.secid, a.exdate,  
							a.cp_flag, a.strike_price, a.forward_price
						FROM
							optionm_all.opprcd{year} AS a
						WHERE
							a.secid=108105;
					"""
		
		optm = db.raw_sql(sql_query, date_cols = ["date"])

		optm_df.append(optm)
		
	db.close()

	optm_df = pd.concat(optm_df)
	
	return optm_df

In [3]:
optm_df = pull_all_option_price("hholt", 2012)

Loading library list...
Done


In [ ]:
optm.shape

In [ ]:
optm_df.head()

,date,secid,exdate,cp_flag,strike_price,forward_price
0,1996-01-05,108105.0,1996-03-16,C,525000.0,None
1,1996-01-05,108105.0,1996-03-16,C,550000.0,None
2,1996-01-09,108105.0,1996-02-17,C,570000.0,None
3,1996-01-09,108105.0,1996-01-20,C,580000.0,None
4,1996-01-17,108105.0,1996-01-20,C,595000.0,None


In [ ]:
optm_df['date'] = pd.to_datetime(optm_df['date'])

In [ ]:
optm_df = optm_df.loc[optm_df['date']<='2012-01-31']

In [ ]:
conn = wrds.Connection(wrds_username="hholt")
conn.list_tables(library="optionm_all")

In [ ]:
conn.get_table(library="optionm_all", table="opprcd1996",obs=5)

In [ ]:
conn.get_table(library="optionm_all", table="stdopd1996",obs=5)

In [ ]:
df = pull_Option_info("hholt")

In [ ]:
opinfd.loc[opinfd['secid']==108105]

In [ ]:
df.head()

In [ ]:
dup_cols = ['cp_flag','strike_price']

duplicates = df[df.duplicated(subset=dup_cols, keep=False)]


In [ ]:
duplicates.head()